In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from sklearn import metrics
import gc
import matplotlib.pyplot as plt
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class data():
    def __init__(self, seq_length, batch_size):
        np.random.seed(100)
        self.seq_length = seq_length
        self.acc_num = 334
        self.no_acc_num = 392
        self.split = 0.95
        self.train_index = 0 
        self.train_batch_size = batch_size
        self.valid_index = 0
        self.valid_batch_size = 12
        self.read_annotation()
        self.shuffle_data()
        
    def shuffle_data(self):
        #img_range = np.arange(0,300-self.seq_length-60,60)
        acc_list = np.arange(1,self.acc_num+1)
        no_acc_list = np.arange(1,self.no_acc_num+1)
        list1 = np.array(np.meshgrid(1,acc_list)).T.reshape(-1,2)
        list2 = np.array(np.meshgrid(0,no_acc_list)).T.reshape(-1,2)
        shuffle_list = np.concatenate([list1, list2], axis=0)
        np.random.shuffle(shuffle_list)
        self.train = shuffle_list[:int(shuffle_list.shape[0]*self.split)]
        self.valid = shuffle_list[int(shuffle_list.shape[0]*self.split):] 
        
    def read_annotation(self):
        annotation_file = '/media/user/Hard_Disk/Dataset/child_accident_2/annotation/accident_frame.txt'
        w = open(annotation_file, "r")
        ann = w.read()
        annotation_data = []
        for i in ann.split("\n"):
            b = i.split(" ")
            if (len(b) > 1):
                annotation_data.append(b[1])
        self.annotation = np.array(annotation_data).astype("int32")
        
    def read_data(self, is_accident, dir_index):
        data = []
        label = []
        range_start = 150 - self.seq_length
        range_end = 150
        img_path = '/media/user/Hard_Disk/Dataset/child_accident_2/image feature/' 
        act_path = '/media/user/Hard_Disk/Dataset/child_accident_2/action feature/' 
        if (is_accident):
            acc_dir = "accident/"
            accident_frame = self.annotation[dir_index-1]
            if (accident_frame > 150 and accident_frame <= 180):
                label.append(np.array([1,0,0,0,0,0]))
            elif (accident_frame > 180 and accident_frame <= 210):
                label.append(np.array([0,1,0,0,0,0]))
            elif (accident_frame > 210 and accident_frame <= 240):
                label.append(np.array([0,0,1,0,0,0]))
            elif (accident_frame > 240 and accident_frame <= 270):
                label.append(np.array([0,0,0,1,0,0]))
            elif (accident_frame > 270 and accident_frame <=300):
                label.append(np.array([0,0,0,0,1,0]))
            else:
                label.append(np.array([0,0,0,0,0,1]))
        else:
            acc_dir = "no_accident/"
            
            label.append(np.array([0,0,0,0,0,1]))
            
        dir_name = "%04d"%dir_index
        img_npy = []
        act_npy = []
        for j in range(range_start, range_end):
            
            img_feature = np.load(img_path + acc_dir + dir_name + "/" + str(j) + ".npy")
            img_npy.append(img_feature)
            act_feature = np.load(act_path + acc_dir + dir_name + "/" + str(j) + ".npy")
            act_npy.append(act_feature)
        
            
        return np.array(img_npy), np.array(act_npy) ,np.array(label)
        
    def next_batch(self, mode="train"):
        batch_img = []
        batch_act = []
        batch_y = []
        if (mode == "valid"):
            batch_size = self.valid_batch_size
        elif (mode == "train"):
            batch_size = self.train_batch_size
        for i in range(batch_size):
            if (mode == "train"):
                img, act, label = self.read_data(self.train[self.train_index+i][0], self.train[self.train_index+i][1])
            elif (mode == "valid"):
                img, act, label = self.read_data(self.valid[self.valid_index+i][0], self.valid[self.valid_index+i][1])
            batch_img.append(img)
            batch_act.append(act)
            batch_y.append(label)
        if (mode == "valid"):
            self.valid_index += self.valid_batch_size
        elif (mode == "train"):
            self.train_index += self.train_batch_size
        return np.array(batch_img), np.array(batch_act), np.squeeze(np.array(batch_y))
    
    def has_next(self, mode="train"):
        if (mode == "train"):
            if (self.train_index + self.train_batch_size >= self.train.shape[0]):
                return False
        elif (mode == "valid"):
            if (self.valid_index + self.valid_batch_size >= self.valid.shape[0]):
                return False
        return True
    def display_shape(self):
        print("train shape:",self.train.shape, " valid shape:",self.valid.shape)
        
    def reset_batch(self, mode="train"):
        if (mode == "train"):
            self.train_index = 0
            np.random.shuffle(self.train)
        elif (mode == "valid"):
            self.valid_index = 0
        
   

In [3]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, img_input_size, act_input_size, hidden_size, embedding_size, embedding_size2, num_layers, num_classes):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.img = nn.Sequential(
            nn.Linear(img_input_size, embedding_size),
            nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(embedding_size, embedding_size2)
        )
        self.act = nn.Sequential(
            nn.Linear(act_input_size, embedding_size),
            nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(embedding_size, embedding_size2)
        )
            
        
        self.img_fc1 = nn.Linear(img_input_size, embedding_size)
        self.act_fc1 = nn.Linear(act_input_size, embedding_size)
        self.img_fc2 = nn.Linear(embedding_size, embedding_size2)
        self.act_fc2 = nn.Linear(embedding_size, embedding_size2)
        self.lstm1 = nn.LSTM(embedding_size2*2, hidden_size, num_layers,dropout=0.25, batch_first=True)
        self.lstm2 = nn.LSTM(embedding_size2*2, hidden_size, num_layers,dropout=0.25, batch_first=True)
        self.lstm3 = nn.LSTM(embedding_size2*2, hidden_size, num_layers,dropout=0.25, batch_first=True)
        self.st3_state_h = nn.Linear(hidden_size*3, hidden_size)
        self.st3_state_c = nn.Linear(hidden_size*3, hidden_size)
        self.st2_state_h = nn.Linear(hidden_size*2, hidden_size)
        self.st2_state_c = nn.Linear(hidden_size*2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x, y):
        # Set initial hidden and cell states 
        x1 = self.img_fc1(x)
        x1 = self.img_fc2(x1)
        x2 = self.act_fc1(y)
        x2 = self.act_fc2(x2)
#         x1 = self.img(x)
#         x = self.act(y)
        x = torch.cat((x1, x2), 2)
#         print(x.shape)
        
        st1_h = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        st1_c = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        st2_h = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        st2_c = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        st3_h = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        st3_c = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        
        for i in range(x.size(1)):
        # Forward propagate LSTM
            stack1_out, (st1_h, st1_c) = self.lstm1(x[:,i].unsqueeze(1), (st1_h, st1_c))  # out: tensor of shape (batch_size, seq_length, hidden_size)
            stack2_out, (st2_h, st2_c) = self.lstm2(x[:,int(i/10)].unsqueeze(1), (st2_h, st2_c))
            stack3_out, (st3_h, st3_c) = self.lstm3(x[:,int(i/20)].unsqueeze(1), (st3_h, st3_c))
           
            
            st3_h = torch.cat((st1_h, st2_h, st3_h), 2)
            st3_h = self.st3_state_h(st3_h)
            
            st3_c = torch.cat((st1_c, st2_c, st3_c), 2)
            st3_c = self.st3_state_c(st3_c)
            
            
            st2_h = torch.cat((st1_h, st2_h), 2)
            st2_h = self.st2_state_h(st2_h)
            
            st2_c = torch.cat((st1_c, st2_c), 2)
            st2_c = self.st2_state_c(st2_c)
            
            
        
        # Decode the hidden state of the last time step
        
        out = self.fc2(stack3_out[:,-1,:])
        
#         out = self.fc2(out[:, -1, :])
        return out

In [4]:
model = RNN(img_input_size=20*4096, act_input_size=1024, hidden_size=1024, embedding_size=1024, embedding_size2=512, num_layers=1, num_classes=6).to(device)

/home/user/.local/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [5]:
w = torch.from_numpy(np.array([1,1,1,1,1,0.5])).type(torch.FloatTensor).to(device)
criterion = nn.CrossEntropyLoss(weight=w)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [6]:
sequence_length = 100
num_epochs = 20
train_data = data(seq_length=sequence_length, batch_size=50)
train_data.display_shape()

train shape: (689, 2)  valid shape: (37, 2)


In [7]:
# out of memory
def test():
    model = torch.load('pytorch - 06/model-90.ckpt')
    while(train_data.has_next("valid")):
        img, act, labels = train_data.next_batch("valid")
        labels = torch.from_numpy(labels).to(device)
        img = torch.from_numpy(img.reshape(-1, sequence_length, 20*4096)).to(device)
        act = torch.from_numpy(np.squeeze(act)).to(device)
        outputs = model(img, act)
        outputs = F.softmax(outputs)
        predict = torch.max(outputs, 1)[1]
        target = torch.max(labels, 1)[1]
        recall(predict, target)
        torch.cuda.empty_cache()

In [8]:
def metric(pred, y):
    pred = np.reshape(np.array(pred.cpu().numpy()), (-1,1))
    y = np.reshape(np.array(y.cpu().numpy()), (-1, 1))
    print(pred.reshape(1,-1))
    print(y.reshape(1,-1))
    print("=========================================")
    print("precision")
    print(metrics.precision_score(y, pred, average='macro'))
    
    print("Accuracy")
    print(metrics.accuracy_score(y, pred))
    
    print("Recall")
    print(metrics.recall_score(y, pred, average='macro'))
    
    print("F1_score")
    print(metrics.f1_score(y, pred, average='weighted'))
    
#     print("roc_auc_score")
#     fpr, tpr, thresholds = metrics.roc_curve(y, pred)
#     roc_auc = metrics.auc(fpr, tpr)
#     plt.title('Receiver Operating Characteristic')
#     plt.plot(fpr, tpr, 'b',
#     label='AUC = %0.2f'% roc_auc)
#     plt.legend(loc='lower right')
#     plt.plot([0,1],[0,1],'r--')
#     plt.xlim([0,1.0])
#     plt.ylim([0,1.0])
#     plt.ylabel('True Positive Rate')
#     plt.xlabel('False Positive Rate')
#     plt.show()

In [9]:
def evaluation(model):
    with torch.no_grad():
        i = 0
        acc = 0
        while(train_data.has_next("valid")):
            img, act, labels = train_data.next_batch("valid")
            labels = torch.from_numpy(labels).to(device)
            img = torch.from_numpy(img.reshape(-1, sequence_length, 20*4096)).to(device)
            act = torch.from_numpy(np.squeeze(act)).to(device)
            outputs = model(img, act)
            outputs = F.softmax(outputs)
            predict = torch.max(outputs, 1)[1]
            target = torch.max(labels, 1)[1]
            loss = criterion(outputs, target)
            correct = (predict == target).squeeze()
            acc += torch.nonzero(correct).size(0) / predict.shape[0]
            i += 1
            metric(predict, target)
        train_data.reset_batch("valid")
        print("validation accuracy:",acc/i)

In [10]:

# Loss and optimizer

# Train the model

for epoch in range(31):
    i = 0
    acc = 0
    print("epoch:",epoch)
    while (train_data.has_next("train")):
        img, act, labels = train_data.next_batch("train")
        labels = torch.from_numpy(labels).to(device)
        img = torch.from_numpy(img.reshape(-1, sequence_length, 20*4096)).to(device)
        act = torch.from_numpy(np.squeeze(act)).to(device)
        outputs = model(img, act)
        outputs = F.softmax(outputs)
#         print(outputs.shape)
#         outputs, torch.max(labels, 1)[1]
        
        predict = torch.max(outputs, 1)[1]
        target = torch.max(labels, 1)[1]
        
        loss = criterion(outputs, target)
        
        
        correct = (predict == target).squeeze()
#         print(predict)
#         print(target)
#         print("================")
#         print(correct)
        acc += torch.nonzero(correct).size(0) / predict.shape[0]
        
    
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        if (i+1) % 5 == 0:
            print ('Epoch [{}/{}], Step {}, Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, loss.item()))
            print("precision")
            print(metrics.precision_score(target, predict, average='macro'))

            print("Accuracy")
            print(metrics.accuracy_score(target, predict))

            print("Recall")
            print(metrics.recall_score(target, predict, average='macro'))

            print("F1_score")
            print(metrics.f1_score(target, predict, average='weighted'))
            
#             print ('train accuracy:',acc/i)
            print("====================================")
    if (epoch % 10 == 0):
        evaluation(model)
        path = "/media/user/Hard_Disk/Dataset/child_accident_2/model/pytorch - 09/model-"+str(epoch)+".ckpt"
        torch.save(model, path)
    train_data.reset_batch("train")
# # Test the model
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.reshape(-1, sequence_length, input_size).to(device)
#         labels = labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# # Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')

epoch: 0


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Epoch [1/20], Step 5, Loss: 1.6190
precision
0.09999999999999999
Accuracy
0.6
Recall
0.16666666666666666
F1_score
0.44999999999999996


/home/user/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/user/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch [1/20], Step 10, Loss: 1.6131
precision
0.15
Accuracy
0.6
Recall
0.25
F1_score
0.44999999999999996


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


[[5 5 5 5 5 5 5 5 5 5 5 5]]
[[5 3 0 2 2 5 5 5 4 5 5 2]]
precision
0.1
Accuracy
0.5
Recall
0.2
F1_score
0.3333333333333333
[[5 5 5 5 5 5 5 5 5 5 5 5]]
[[5 4 5 5 3 5 2 5 5 5 1 1]]
precision
0.11666666666666667
Accuracy
0.5833333333333334
Recall
0.2
F1_score
0.4298245614035088
[[5 5 5 5 5 5 5 5 5 5 5 5]]
[[5 5 5 2 3 5 4 5 5 2 5 2]]
precision
0.14583333333333334
Accuracy
0.5833333333333334
Recall
0.25
F1_score
0.4298245614035088
validation accuracy: 0.5555555555555557


/home/user/.local/lib/python3.5/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


epoch: 1
Epoch [2/20], Step 5, Loss: 1.6865
precision
0.08666666666666667
Accuracy
0.52
Recall
0.16666666666666666
F1_score
0.35578947368421054
Epoch [2/20], Step 10, Loss: 1.7085
precision
0.092
Accuracy
0.46
Recall
0.2
F1_score
0.28986301369863016
epoch: 2
Epoch [3/20], Step 5, Loss: 1.5936
precision
0.09999999999999999
Accuracy
0.6
Recall
0.16666666666666666
F1_score
0.44999999999999996
Epoch [3/20], Step 10, Loss: 1.7005
precision
0.08
Accuracy
0.48
Recall
0.16666666666666666
F1_score
0.3113513513513514
epoch: 3
Epoch [4/20], Step 5, Loss: 1.7094
precision
0.08
Accuracy
0.48
Recall
0.16666666666666666
F1_score
0.3113513513513514
Epoch [4/20], Step 10, Loss: 1.6990
precision
0.08333333333333333
Accuracy
0.5
Recall
0.16666666666666666
F1_score
0.33333333333333326
epoch: 4
Epoch [5/20], Step 5, Loss: 1.6248
precision
0.11599999999999999
Accuracy
0.58
Recall
0.2
F1_score
0.42582278481012653
Epoch [5/20], Step 10, Loss: 1.6090
precision
0.09666666666666666
Accuracy
0.58
Recall
0.1666666

Epoch [22/20], Step 10, Loss: 1.4287
precision
0.2916943521594685
Accuracy
0.74
Recall
0.34285714285714286
F1_score
0.6461333333333333
epoch: 22
Epoch [23/20], Step 5, Loss: 1.4323
precision
0.22525252525252526
Accuracy
0.7
Recall
0.3088624338624339
F1_score
0.6157377049180328
Epoch [23/20], Step 10, Loss: 1.4191
precision
0.23214285714285712
Accuracy
0.72
Recall
0.31666666666666665
F1_score
0.6128571428571429
epoch: 23
Epoch [24/20], Step 5, Loss: 1.4345
precision
0.2191142191142191
Accuracy
0.72
Recall
0.3333333333333333
F1_score
0.6064450127877238
Epoch [24/20], Step 10, Loss: 1.3778
precision
0.30421146953405015
Accuracy
0.72
Recall
0.34490740740740744
F1_score
0.6371688311688312
epoch: 24
Epoch [25/20], Step 5, Loss: 1.5013
precision
0.2560656189688448
Accuracy
0.62
Recall
0.35909090909090907
F1_score
0.5141337907375643
Epoch [25/20], Step 10, Loss: 1.5790
precision
0.24444444444444444
Accuracy
0.54
Recall
0.30277777777777776
F1_score
0.5161904761904762
epoch: 25
Epoch [26/20], St